In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
ls expts/rel3_tsel_ref_dial_model_separate/directions

bad_filtered/
base_512_bsz-64_lr-5e-3_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_bsz-64_rel-2_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm-lPm-lPpm_prb-lpm-lm-lPm-lPpm/
base_512_extremes_asymmetric_learned-pooling_untied_bidirectional_hidden-mention-encoder_rb-lpm-lm_prb-lpm-lm/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_2/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder_3/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/
base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-fil

In [5]:
ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

 1_best.stdict
 1_best.th
 1_ep-30.stdict
 1_ep-30.th
'eval_dev_cand=40_oracle.out'
'eval_dev_cand=40_speaker-weight=0.1.out'
'eval_dev_cand=40_speaker-weight=0.2.out'
'eval_dev_cand=40_speaker-weight=0.3.out'
'eval_dev_cand=40_speaker-weight=0.4.out'
'eval_dev_cand=40_speaker-weight=0.5.out'
'eval_dev_cand=40_speaker-weight=0.6.out'
'eval_dev_cand=40_speaker-weight=0.7.out'
'eval_dev_cand=40_speaker-weight=0.8.out'
'eval_dev_cand=40_speaker-weight=0.9.out'
'eval_dev_cand=40_speaker-weight=0.out'
'eval_dev_cand=40_speaker-weight=1.0.out'
 eval_dev_l0.out
 eval_dev_l0_ref_annotations.json
 train_fold-1.out


In [6]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'
dot_recurrence = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
    'encoder-filtered-separate-diffs': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/1_ep-30.th',
    'dot-recurrence': f'{dot_recurrence}/base_dr-self+partner-32-mention-selection/1_ep-20.th',
    'dot-recurrence-nm-multi': f'{dot_recurrence}/base_dr-self+partner-64-mention-selection_nmpt-multi-reference_expanded/1_ep-20.th',
}

In [7]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 10
vocab size: 649
dataset data/onecommon/train_reference_1.txt, total 545056, unks 4302, ratio 0.79%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 686, ratio 0.99%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 636, ratio 0.95%


In [8]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [9]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [10]:
len(scenarios1)

9000

In [11]:
len(scenarios2)

4500

In [12]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [13]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [14]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [15]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [16]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [17]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [18]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:01<00:00, 835.67it/s]


In [19]:
validset_stats

{'n': 74030,
 'self_n': 37015,
 'nonpadn': 69348,
 'self_nonpadn': 34674,
 'max_num_mentions': 9}

In [20]:
import torch

In [21]:
from agent import RnnAgent

In [22]:
from selfplay import make_parser

In [23]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [24]:
def is_selection(out):
    return '<selection>' in out

In [25]:
# display

In [26]:
from dialog import DialogLogger

In [27]:
from IPython.display import SVG, display, HTML

In [28]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [29]:
SVG_SCALE=0.4

In [30]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [31]:
def display_attns(scenario, attns, agent_id, name=None):
    svgs = [DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id]]
#     if name is not None:
#         print("{}: {}".format(name, attn))
    for attn in attns:
        attn = attn.flatten().detach().numpy()
        svgs.append(DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE))
    if name is not None:
        print(name)
    display_svgs(svgs)

In [32]:
# training emulation

In [33]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:11<00:00, 693.92it/s]


In [34]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [35]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [36]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [37]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate', 'encoder-filtered-separate-diffs', 'dot-recurrence', 'dot-recurrence-nm-multi'])

In [38]:
# model = models['encoder-filtered-separate-diffs']
# model = models['dot-recurrence']
model = models['dot-recurrence-nm-multi']

In [39]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [40]:
from models.reference_predictor import ReferencePredictor

In [41]:
from engines.beliefs import BeliefConstructor

In [42]:
from engines.rnn_reference_engine import make_dots_mentioned_multi, make_dots_mentioned_per_ref_multi

In [43]:
reference_predictor = ReferencePredictor(model.args)

In [44]:
pdb on

Automatic pdb calling has been turned ON


In [86]:
def force_contexts_and_mentions(index, datasplit=validset, samples=10, sample_temperature=0.25, argmax_temperature=0.005, force_num_markables=True):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    bsz = ctx.size(0)
    num_dots = 7
    
    # assume we've been training for a while, so that the beliefs are out of the warmup period (e.g. for models that switch from true->predicted)
    epoch = 1000
    
    dots_mentioned = make_dots_mentioned_multi(ref_tgts, model.args, bsz, num_dots)
    dots_mentioned_per_ref = make_dots_mentioned_per_ref_multi(ref_tgts, model.args, bsz, num_dots)

    partner_dots_mentioned_our_view = make_dots_mentioned_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    partner_dots_mentioned_our_view_per_ref = make_dots_mentioned_per_ref_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    
    belief_constructor = BeliefConstructor(
        model.args, epoch, bsz, num_dots, inpts, ref_tgts, partner_ref_tgts_our_view,
        real_ids, partner_real_ids, sel_tgt, is_self,
        partner_dots_mentioned_our_view, partner_dots_mentioned_our_view_per_ref,
        dots_mentioned, dots_mentioned_per_ref,
        ref_inpts, partner_ref_inpts,
        num_markables, partner_num_markables,
    )
    
    scenario = all_scenarios[scenario_id]
    agent = alice
    agent.feed_context(ctx.flatten(),
                       belief_constructor,
                       num_markables_to_force=num_markables[0] if force_num_markables else None)
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    writer = alice
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
        
        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
            mentions = [dots_mentioned_per_ref[:,mention_ix].float()  for mention_ix in range(dots_mentioned_per_ref.size(1))]
            display_attns(scenario, mentions, writer.agent_id, "to_mention_true")
            nm_out, _, nm_num_markables = writer.next_mention_outs[-1]
#             print('nm_num_markables: {}'.format(nm_num_markables))
#             print('num_markables: {}'.format(this_num_markables))
            if nm_out is not None:
#                 print(nm_out[0].size())
                _, nm_preds, _ = reference_predictor.forward(
                    True, dots_mentioned_per_ref, nm_out, nm_num_markables
                )
#                 print(nm_preds.size())
                display_attns(scenario, nm_preds.squeeze(1).float(), writer.agent_id, "to_mention_pred")
#             for mention_ix in range(dots_mentioned_per_ref.size(1)):
#                 display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            for sample_ix in range(samples):
                pred_outs, _, _, _, extra = writer.model.write(
                    writer.state, 
                    words_left, sample_temperature,
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables
                )
                print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
            pred_outs, _, _, _, extra = writer.model.write(
                writer.state, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('argmax\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

            out = writer.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
                ref_inpt=ref_inpts[sentence_ix],
            )
        
    #         if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
    #             display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')

    #         if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
    #             display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')

    #         for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
    #             display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))

        else:
            out = words_og
            writer.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables,
                        partner_ref_inpt=partner_ref_inpts[sentence_ix],
                        partner_num_markables=partner_num_markables[sentence_ix],
                        next_num_markables_to_force=num_markables[sentence_ix + 1] if (force_num_markables and sentence_ix < len(num_markables) - 1) else None,
                       )
        
#         writer.update_dot_h(ref_inpts[sentence_ix], partner_ref_inpts[sentence_ix],
#                             num_markables[sentence_ix], partner_num_markables[sentence_ix])

        print('human\t{}'.format(' '.join(out)))
    
        if is_self[sentence_ix]:
            if ref_inpts[sentence_ix] is not None:
                _, ref_preds, ref_stats = reference_predictor.forward(
                    ref_inpts[sentence_ix], ref_tgts[sentence_ix], writer.ref_outs[-1], this_num_markables
                )
#                 print("ref_preds size:", ref_preds.size())
                display_attns(scenario, ref_tgts[sentence_ix].squeeze(0).float(), writer.agent_id, "refs_true")
                display_attns(scenario, ref_preds.squeeze(1).float(), writer.agent_id, "refs_pred")
#                 print(ref_stats)
        else:
            if partner_ref_inpts[sentence_ix] is not None:
                _, partner_ref_preds, partner_ref_stats = reference_predictor.forward(
                    partner_ref_inpts[sentence_ix], partner_ref_tgts_our_view[sentence_ix],
                    writer.partner_ref_outs[-1], partner_num_markables[sentence_ix]
                )
    #             print(partner_ref_preds)
                display_attns(scenario, partner_ref_tgts_our_view[sentence_ix].squeeze(0).float(), writer.agent_id, "partner_refs_true")
                display_attns(scenario, partner_ref_preds.squeeze(1).float(), writer.agent_id, "partner_refs_pred")
#                 print(partner_ref_stats)

        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out) or words_left <= 1:
            break
        sentence_ix += 1

In [75]:
pdb on

Automatic pdb calling has been turned ON


In [87]:
force_contexts_and_mentions(1)

to_mention_true


to_mention_pred


sample 0	i have a small dark grey dot by itself <eos>
sample 1	i have a small black dot by itself <eos>
sample 2	i have a small black dot by itself <eos>
sample 3	i have a small black dot by itself <eos>
sample 4	i have a small black dot by itself <eos>
sample 5	i have a small black dot by itself <eos>
sample 6	i have a small black dot by itself <eos>
sample 7	i have a tiny black dot by itself <eos>
sample 8	i have a small black dot by itself <eos>
sample 9	i have a small dark gray dot by itself <eos>
argmax	i have a small black dot by itself <eos>
human	tiny dark dot all by itself ? <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: single larger light grey dot <eos>
partner_refs_true


...


----------------------------------------

human	THEM: yes <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	ok i have that one . <selection>
sample 1	ok , lets choose that one <selection>
sample 2	ok , lets click it <selection>
sample 3	ok , let 's pick that one . <selection>
sample 4	ok , it is <selection>
sample 5	do you have a large light grey dot ? <eos>
sample 6	ok . let 's try it <selection>
sample 7	okay , let 's choose that one <selection>
sample 8	ok , let 's try it <selection>
sample 9	ok , let 's pick it <selection>
argmax	ok , let 's choose that one <selection>
human	select it then ! <selection>
refs_true


refs_pred



----------------------------------------



In [88]:
force_contexts_and_mentions(2)

human	THEM: hi , i see a gray medium size circle at the bottom edge of my <unk> <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i have a triangle of dots , one is small and black <eos>
sample 1	i have a triangle of dots , the one on the top is the largest and darkest <eos>
sample 2	i have a triangle of dots , one is in the middle of the other <eos>
sample 3	i see a triangle of dots . the one on the left is the largest and darkest <eos>
sample 4	i have a group of three dots that form a triangle , the top one is the darkest <eos>
sample 5	i see a line of 3 dots , the top one is darker and smaller <eos>
sample 6	i have two dots that are almost same size and size . the one on the right is a small darker dot <eos>
sample 7	i have a triangle of dots . one is on the left of the circle <eos>
sample 8	i have three dots that are almost in a vertical line . the one on the bottom is the largest and darkest <eos>
sample 9	i have a triangle of dots , one is dark grey and the other is medium grey <eos>
argmax	i have a triangle of dots , the one on the right is the largest and darkest <eos>
human	i have three darkish 

...


----------------------------------------

human	THEM: ok <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	lets pick the larger one <selection>
sample 1	let 's pick the larger one . <selection>
sample 2	let 's pick the larger one . <eos>
sample 3	let 's pick the large dark grey one . <selection>
sample 4	let 's choose the large one on the right . <selection>
sample 5	let 's pick the larger one on the right <selection>
sample 6	let 's pick the large one <selection>
sample 7	let 's pick the larger one <selection>
sample 8	let 's pick the larger one . <eos>
sample 9	let 's pick the larger one on the right . <selection>
argmax	let 's pick the larger one . <selection>
human	let 's click on the largest one <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [89]:
force_contexts_and_mentions(3)

to_mention_true


to_mention_pred


sample 0	i have a small black dot , with a slightly larger dark gray dot down and to the right <eos>
sample 1	i have one small black dot , with a slightly larger black dot to the right and slightly above <eos>
sample 2	i have a small black dot , with a slightly larger black dot to the left <eos>
sample 3	i have a small black dot , and to the right of a larger black dot <eos>
sample 4	i have a small black dot , and to the right of a larger black dot <eos>
sample 5	i have a small black dot , with a larger black dot to the right <eos>
sample 6	i have a small black dot , with a larger black dot to the left <eos>
sample 7	i have a small black dot to the right of a larger black dot <eos>
sample 8	i have a tiny black dot to the right of a larger black dot <eos>
sample 9	i have a small black dot to the left of a larger dark dot <eos>
argmax	i have a small black dot to the left of a larger black dot <eos>
human	small black dot alone on the right ... larger black dot alone on the left <eos>
refs

refs_pred



----------------------------------------

human	THEM: hmm i do n't see that . i see three dots almost in a horizontal line with <unk> one largest and darker , middle small and very light grey , and medium size medium color grey on right <eos>
partner_refs_true


...


----------------------------------------

to_mention_true


to_mention_pred


sample 0	yes , let 's choose the black one . <selection>
sample 1	yes , lets choose the black one <eos>
sample 2	yes , choose the black one <selection>
sample 3	yes . let 's choose the black one . <selection>
sample 4	yes , let 's pick the darker one <eos>
sample 5	yes , let 's choose the top one <eos>
sample 6	yes , lets choose the darker one <selection>
sample 7	yes , let 's pick the darker one <selection>
sample 8	yes , let 's pick the darker one <selection>
sample 9	yes , let 's pick the darker one <eos>
argmax	yes , let 's pick the darker one <eos>
human	go with the bottom darkest one <selection>
refs_true


refs_pred



----------------------------------------



In [90]:
force_contexts_and_mentions(4)

to_mention_true


to_mention_pred


sample 0	i see a triangle of dots <eos>
sample 1	i have a medium size medium grey dot in the middle <eos>
sample 2	i have a medium gray dot at the top <eos>
sample 3	do you see a medium grey dot alone ? <eos>
sample 4	do you see a medium grey dot in the middle of the circle <eos>
sample 5	two dots very close together ? <eos>
sample 6	i have a medium size , medium gray dot at the bottom of the circle <eos>
sample 7	i have three dots that are almost in a horizontal line <eos>
sample 8	i have a medium gray dot at the top <eos>
sample 9	i have a medium sized medium gray dot at the bottom <eos>
argmax	i have a medium gray dot at the top <eos>
human	i see a group of 3 about the same shade , on the left <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: is there a black <unk> one below them or did that not make it on screen ? <eos>
partner_refs_true


...

sample 0	yes . it is on the left of the circle and there is a small light grey dot to the right of it <eos>
sample 1	yes . is the one on the left of the two dark grey dots ? if so click the black one ! <selection>
sample 2	yes , it is also a little darker . i have a large dark grey dot and a smaller black dot <eos>
sample 3	yes , the black one is the smallest . i have a small light grey dot with a slightly smaller dark grey dot to the left and slightly above it <eos>
sample 4	yes . it is to the right of the large dot . is there a small light grey dot to the left and slightly above ? <eos>
sample 5	yes . is there a large black dot to the right of the small light grey dot ? if so click the black one ! <eos>
sample 6	yes , it 's a little darker . with a small light grey one to its lower left ? <eos>
sample 7	yes . it 's the darkest one , there is a small light grey dot above it <eos>
sample 8	yes , it is to the right of the circle . is there a small light grey dot to the right of the blac

refs_pred



----------------------------------------

human	THEM: yep , take the black one <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [91]:
force_contexts_and_mentions(5)

human	THEM: i see a large gray dot on the top on the dotted line <eos>
partner_refs_true


partner_refs_pred



----------------------------------------

to_mention_true


to_mention_pred


sample 0	i have a triangle of 3 dots . one large and light grey , one medium gray , one small and light grey <eos>
sample 1	i have two dots very close together . the one on the left is a small dark grey dot , and the one on the right is a little larger , lighter dot . <eos>
sample 2	i have two dots very close together . the one on the right is smaller , the one on the left is small and light grey . the one on the right is slightly larger and darker <eos>
sample 3	i have two dots very close together . one is smaller and darker than the other . <eos>
sample 4	i have two dots very close together . one is dark grey and the other is dark grey and above it and to the right <eos>
sample 5	i have two dots very close together . one is black and one light grey , and one small light grey <eos>
sample 6	i have two dots very close together . one is darker than the other . <eos>
sample 7	i have two dots very close together . the one on the right is a small dark dot , and the one on the right is a li

...


----------------------------------------

to_mention_true


to_mention_pred


sample 0	pick the larger one <selection>
sample 1	let 's pick the larger one <selection>
sample 2	lets pick the larger one <selection>
sample 3	let 's pick the larger one <selection>
sample 4	let 's pick the large grey one <selection>
sample 5	let 's choose the larger one <selection>
sample 6	let 's pick the larger one <selection>
sample 7	lets pick the larger one <selection>
sample 8	let 's pick the large grey dot <selection>
sample 9	lets pick the larger one <selection>
argmax	let 's pick the larger one <selection>
human	let 's pick the larger one on the right <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: okay <selection>

----------------------------------------



In [92]:
force_contexts_and_mentions(6)

to_mention_true


to_mention_pred


sample 0	i have a cluster of dots forming a triangle <eos>
sample 1	i have a cluster of 4 dots <eos>
sample 2	i see a cluster of dots . <eos>
sample 3	do you see a cluster of dots in a line ? <eos>
sample 4	i see a triangle of dots in a line going up and to the right <eos>
sample 5	i see a cluster of dots . <eos>
sample 6	do you see a cluster of dots that are all the same size and a small dot ? <eos>
sample 7	i have a cluster of 4 dots <eos>
sample 8	i have a cluster of 4 dots <eos>
sample 9	do you see a cluster of dots that form a triangle shape ? <eos>
argmax	i see a cluster of dots . <eos>
human	a cluster of 4 dots very close together ? <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: 2 small black ones and 2 larger grey ones ? <eos>
partner_refs_true


...


----------------------------------------

human	THEM: yes <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	ok , lets click the larger one <selection>
sample 1	ok , lets click the larger one <selection>
sample 2	lets pick the larger one <selection>
sample 3	ok , lets pick the larger one <selection>
sample 4	k click the large light one on top ! <selection>
sample 5	let 's choose the larger one on the right <selection>
sample 6	lets pick the large one <selection>
sample 7	ok , lets click the larger one <selection>
sample 8	lets pick the larger one <selection>
sample 9	ok , lets click the larger one <selection>
argmax	ok , lets click the larger one on the right <selection>
human	yes click the large very light grey one all the way to the right <selection>
refs_true


refs_pred



----------------------------------------



In [93]:
force_contexts_and_mentions(7)

to_mention_true


to_mention_pred


sample 0	i have a large light grey dot by itself <eos>
sample 1	i have a large light gray dot by itself <eos>
sample 2	i have a large light grey dot by itself <eos>
sample 3	i have a large light gray dot by itself <eos>
sample 4	i have a large light grey dot by itself <eos>
sample 5	i have a large light gray dot by itself <eos>
sample 6	i have a large light gray dot by itself <eos>
sample 7	i have a large light grey dot by itself <eos>
sample 8	i have a large light gray dot by itself <eos>
sample 9	do you see a large light grey dot by itself ? <eos>
argmax	i have a large light grey dot by itself <eos>
human	there is a large light grey dot at the bottom center do u have it <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: yes <eos>

----------------------------------------

to_mention_true


to_mention_pred


sample 0	let 's pick that one <selection>
sample 1	let 's pick that one <selection>
sample 2	let 's pick that one <selection>
sample 3	pick the large light grey dot <selection>
sample 4	ok let 's pick that one <selection>
sample 5	lets pick that one <selection>
sample 6	pick the large light grey dot <selection>
sample 7	let 's choose that one <selection>
sample 8	let 's select that one <selection>
sample 9	let 's choose that one <selection>
argmax	let 's pick that one <selection>
human	lets click it <selection>
refs_true


refs_pred



----------------------------------------



In [94]:
force_contexts_and_mentions(8)

to_mention_true


to_mention_pred


sample 0	i have a large black dot at the bottom of the circle <eos>
sample 1	i have a large black dot <eos>
sample 2	i have a large black dot in the middle of the circle <eos>
sample 3	i have a large black dot at the top of the circle <eos>
sample 4	i have a large black dot <eos>
sample 5	i have one large black dot <eos>
sample 6	i have a large black dot <eos>
sample 7	i have a large black dot <eos>
sample 8	i have a large black dot in the middle of the circle <eos>
sample 9	large black dot ? <eos>
argmax	i have a large black dot <eos>
human	i have a <unk> large black dot <eos>
refs_true


refs_pred



----------------------------------------

human	THEM: i have no large dots , i have two dots close to each other , one medium dark grey , one small light grey to its right <eos>
partner_refs_true


...

sample 0	i have a large black dot , and a light grey dot to the left of it <eos>
sample 1	i see a dark gray dot , with a light grey dot to the right of it . <eos>
sample 2	i think i see that . is there a lighter one above and to the right of the smaller one ? <eos>
sample 3	i have a large black dot and a small light grey dot to the right of it <eos>
sample 4	i have one large black dot , above it and to the right is a small light grey dot <eos>
sample 5	i have a dark grey dot , with a lighter grey dot above and to the left of it . <eos>
sample 6	i have two dots that are very close together . the one on the left is slightly smaller and darker and a little higher than the one on the right <eos>
sample 7	i have a medium black dot with a light grey dot to the right of it <eos>
sample 8	i see a black dot that is above and to the right of a light grey dot . <eos>
sample 9	i have two dark dots , one is slightly larger than the other . <eos>
argmax	i have two dark dots . one is slightly larger 

refs_pred



----------------------------------------

human	THEM: yes , pick the small light grey dot <selection>
partner_refs_true


partner_refs_pred



----------------------------------------



In [54]:
force_contexts_and_mentions(9)

human	THEM: do you see one very large black dot ? it 's the only dark dark one <eos>

----------------------------------------



sample 0	i have a large black dot , and to the right of it is a smaller light grey dot , and to the right of it is a smaller light grey dot . i have a cluster of 3 dots <eos>
sample 1	i have a large black dot . above and to the right is a smaller grey dot , and below that black dot is a medium gray dot and to the right of it is a small light dot <eos>
sample 2	i have a large black dot , and to the right of it is a small light dot , and below that black dot is a medium grey dot . <eos>
sample 3	i have a large black dot . above it and to the right is a small light grey dot , and to the left of it is a smaller black dot . below that black dot is a light grey dot <eos>
sample 4	i have a large black dot . above and to the left is a small light grey dot , below and to the right of it is a small light dot , and above that medium grey dot is a medium gray dot <eos>
sample 5	i have a large black dot . below it and to the right is a small light grey dot , and to the right of it is a smaller dark

In [55]:
force_contexts_and_mentions(10)

sample 0	two tiny dots close to each other , one on the right is slightly darker and slightly larger <eos>
sample 1	i have two small dots very close to each other . the one on the right is slightly darker and slightly smaller . <eos>
sample 2	i have two tiny dots close to each other , one is slightly darker and slightly lower <eos>
sample 3	two tiny dots almost vertical to each other ? one on the right is darker and slightly larger <eos>
sample 4	i have two small dots almost on a vertical line . the one on the left is slightly darker and slightly larger . <eos>
sample 5	i have two small dots very close to each other . the one on the right is slightly darker and a little darker <eos>
sample 6	i have two tiny dots close to each other . the one on top is slightly to the right and slightly darker <eos>
sample 7	two small dots , one is slightly darker and to the right of the other ? <eos>
sample 8	i have two small dots close to each other . the one on the right is slightly darker , and a li

sample 0	ok , let 's do it ! <selection>
sample 1	ok , lets click the tiny one <selection>
sample 2	ok , lets click the smaller one <selection>
sample 3	ok , the one on the right is a little darker . <eos>
sample 4	ok , let 's do the one on the right . <selection>
sample 5	ok , lets pick the top one <selection>
sample 6	ok , lets click the top one <selection>
sample 7	ok , lets click the top one <selection>
sample 8	ok , lets click the top one ? <eos>
sample 9	ok , the one on the right is slightly darker . <eos>
argmax	ok , lets click the smaller one <selection>
human	okay i will click the top one . <selection>

----------------------------------------



In [56]:
force_contexts_and_mentions(11)

sample 0	i see two dots very close to each other . <eos>
sample 1	i have two dots of same size and color dots close to each other <eos>
sample 2	two dots very close together and diagonal to each other ? <eos>
sample 3	i have two dots very close to each other , same size <eos>
sample 4	i have two dark dots close to each other <eos>
sample 5	i have two dark dots close to each other <eos>
sample 6	i have two dots that are very close together . they are both medium size <eos>
sample 7	i have two medium sized dots very close to each other <eos>
sample 8	i have two dots of similar size and color dots close to each other <eos>
sample 9	i have two dots very close to each other , very close together <eos>
argmax	i have two dots very close to each other <eos>
human	two medium sized dots really close to each other ? <eos>

----------------------------------------

human	THEM: i have many black dot , but i have one set of medium grey the one on the right is slightly larger and darker ? <eos>

----

sample 0	yes i see that . choose the smaller one <selection>
sample 1	yes i see that . pick the smaller one <selection>
sample 2	yes i have that . let 's pick the smaller one <eos>
sample 3	yes i see that . choose the smaller one <selection>
sample 4	yes , i have that . let 's pick the smaller one <selection>
sample 5	yes i see that . pick the smaller one <selection>
sample 6	yes i see that . choose the smaller one <selection>
sample 7	yes i see that . pick the smaller one <selection>
sample 8	yes i see that . choose the smaller one <selection>
sample 9	yes i see that . choose the smaller one <selection>
argmax	yes i see that . choose the smaller one <selection>
human	i have that ... the left one is a little higher up . <eos>

----------------------------------------

human	THEM: yes . let 's choose that one ... the smaller one on the left . = ) <selection>

----------------------------------------



In [72]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--
